In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !ls "/content/drive/My Drive/ds_ai"

In [3]:
# !pip3 install scikit-learn

In [4]:
# !pip install textblob

In [5]:
from textblob import TextBlob

In [6]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')
import sklearn

from sklearn import feature_selection
# from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers import *
# from keras.layers.embeddings import Embedding
# from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Егор\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
train_df = pd.read_json(r'E:\\CL_Cup\ranking_train.jsonl', lines=True)

In [9]:
df = train_df.explode('comments', ignore_index=True)
df.head(10)

,text,comments
0,How many summer Y Combinator fundees decided n...,{'text': 'Going back to school is not identica...
1,How many summer Y Combinator fundees decided n...,{'text': 'There will invariably be those who d...
2,How many summer Y Combinator fundees decided n...,{'text': 'For me school is a way to be connect...
3,How many summer Y Combinator fundees decided n...,{'text': 'I guess it really depends on how hun...
4,How many summer Y Combinator fundees decided n...,{'text': 'I know pollground decided to go back...
5,CBS acquires last.fm for $280m,{'text': 'It will be curious to see where this...
6,CBS acquires last.fm for $280m,{'text': 'Does this mean that there's now a bi...
7,CBS acquires last.fm for $280m,{'text': 'Also on BBC News: http://news.bbc.c...
8,CBS acquires last.fm for $280m,{'text': 'I don't understand what they do that...
9,CBS acquires last.fm for $280m,{'text': 'sold out too cheaply. given their le...


In [10]:
df_comments = pd.json_normalize(df['comments'])
df_comments.head(5)

,text,score
0,Going back to school is not identical with giv...,0
1,There will invariably be those who don't see t...,1
2,For me school is a way to be connected to what...,2
3,I guess it really depends on how hungry you ar...,3
4,I know pollground decided to go back to school...,4


In [11]:
map_dict = {0: 0.8, 1: 0.65, 2: 0.5, 3: 0.35, 4: 0.2}

df_comments['score'] = df_comments['score'].map(map_dict)
df_comments.columns = ['comment', 'score']
df_comments.tail(20)

,comment,score
440515,We use a HSA-qualified high-deductible policy ...,0.80
440516,I&#x27;m self-employed and use a catastrophic ...,0.65
440517,"Mom &amp; School. Thanks Obama! No really, t...",0.50
440518,I&#x27;ve had one employer offer to pay you th...,0.35
440519,"I ended up on disability in 2003, been on Medi...",0.20
440520,neat insight! A friend of mine convinced me to...,0.80
440521,The fixed header and footer on this page makes...,0.65
440522,"Good read, thanks for putting this together. G...",0.50
440523,Awesome article. Thanks Justin!,0.35
440524,dopeness - very useful,0.20


In [12]:
df = pd.merge(df['text'], df_comments, left_index=True, right_index=True)
df.head(10)

,text,comment,score
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0.80
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,0.65
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,0.50
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,0.35
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,0.20
5,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,0.80
6,CBS acquires last.fm for $280m,Does this mean that there's now a big-name com...,0.65
7,CBS acquires last.fm for $280m,Also on BBC News: http://news.bbc.co.uk/1/low...,0.50
8,CBS acquires last.fm for $280m,I don't understand what they do that is worth ...,0.35
9,CBS acquires last.fm for $280m,sold out too cheaply. given their leadership p...,0.20


In [13]:
def to_wordlist(text, remove_stopwords=True):
    new_text = re.sub("[^a-zA-Z]"," ", text)
    words = new_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = english_stemmer #PorterStemmer()
    for word in words:
        b.append(stemmer.stem(word))
    return b

In [14]:
train, test = train_test_split(df, test_size = 0.2)

In [15]:
clean_train_text = []
for txt in train['text']:
    clean_train_text.append( " ".join(to_wordlist(txt)))
    
clean_test_text = []
for txt in test['text']:
    clean_test_text.append( " ".join(to_wordlist(txt)))

clean_train_comment = []

for txt in train['comment']:
    clean_train_comment.append( " ".join(to_wordlist(txt)))
    
clean_test_comment = []
for txt in test['comment']:
    clean_test_comment.append( " ".join(to_wordlist(txt)))

In [16]:
vectorizer = TfidfVectorizer( min_df=2, max_df=0.95, max_features = 200000, ngram_range = ( 1, 4 ),
                              sublinear_tf = True )

vectorizer = vectorizer.fit(clean_train_comment + clean_train_text)
train_features_t = vectorizer.transform(clean_train_text)
train_features_c = vectorizer.transform(clean_train_text)

test_features_t = vectorizer.transform(clean_test_text)
test_features_c = vectorizer.transform(clean_test_text)

In [17]:
# fselect = univariate_selection.SelectKBest(univariate_selection.chi2 , k=10000)
# train_features = fselect.fit_transform(train_features, train["score"])
# test_features = fselect.transform(test_features)

In [18]:
def sentiment_texblob_p(row):
  
    classifier = TextBlob(row)
    polarity = classifier.sentiment.polarity
    subjectivity = classifier.sentiment.subjectivity
    
    return polarity
def sentiment_texblob_s(row):
  
    classifier = TextBlob(row)
    polarity = classifier.sentiment.polarity
    subjectivity = classifier.sentiment.subjectivity
    
    return subjectivity

In [19]:
df['text_pol'] = df['text'].apply(lambda x: sentiment_texblob_p(x))
df['text_subj'] = df['text'].apply(lambda x: sentiment_texblob_s(x))

In [20]:
df['comment_pol'] = df['comment'].apply(lambda x: sentiment_texblob_p(x))
df['comment_subj'] = df['comment'].apply(lambda x: sentiment_texblob_s(x))

In [21]:
df.head(20)

,text,comment,score,text_pol,text_subj,comment_pol,comment_subj
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0.80,0.25,0.25,-0.042308,0.447436
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,0.65,0.25,0.25,0.172222,0.325000
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,0.50,0.25,0.25,0.190303,0.408788
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,0.35,0.25,0.25,0.176190,0.509524
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,0.20,0.25,0.25,0.000000,0.000000
5,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,0.80,0.00,0.00,0.250000,0.468750
6,CBS acquires last.fm for $280m,Does this mean that there's now a big-name com...,0.65,0.00,0.00,-0.156250,0.468750
7,CBS acquires last.fm for $280m,Also on BBC News: http://news.bbc.co.uk/1/low...,0.50,0.00,0.00,0.000000,0.000000
8,CBS acquires last.fm for $280m,I don't understand what they do that is worth ...,0.35,0.00,0.00,0.300000,0.100000
9,CBS acquires last.fm for $280m,sold out too cheaply. given their leadership p...,0.20,0.00,0.00,0.050000,0.550000


In [22]:
Y=df['score']
X=df.drop(['score', 'text', 'comment'], axis = 1 )
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test  = train_test_split(X, Y, test_size = 0.3, shuffle = True)

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [24]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression() 				#Создаем модель
LR.fit(X_train, Y_train)				#Обучаем модель
Y_LR = LR.predict(X_test)				#Предсказываем значения для выборки    
print ('MAE:', round (mean_absolute_error(Y_test, Y_LR),3))		#Метрики
print ('√MSE:', round (mean_squared_error(Y_test, Y_LR)**(1/2),3))
print ('R2_score:', round (r2_score(Y_test, Y_LR),3))

MAE: 0.181
√MSE: 0.212
R2_score: 0.003
